In [13]:
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import os
import cv2
import keras
import warnings
import time
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import random

In [57]:
# Load a pretrained ResNet-50 model
model = models.resnet50()
model.fc = nn.Linear(model.fc.in_features, 10)
# model.eval()  # Set the model to evaluation mode

In [15]:
def loading_training_data(data_dir):
    data = []
    labels_list = []
    
    for label in labels:
        # path = os.path.join(data_dir, label)
        path = data_dir+"/"+label
        class_num = labels.index(label)
        for img in os.listdir(path):
            # print(img)
            # print(path)
            # img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_arr = cv2.imread(os.path.join(path, img))
            
            # print(os.path.join(path,img))
            if img_arr is None:
                # print(f"Warning: Unable to load image {img}")
                print(os.path.join(path,img))
                continue  # Skip the image if it can't be loaded
            resized_arr = cv2.resize(img_arr, (img_size, img_size))
            data.append(resized_arr)
            labels_list.append(class_num)
            
    return np.array(data), np.array(labels_list)


In [16]:
# Labels for image categories
labels = ['Beagle', 'Boxer', 'Bulldog', 'Dachshund', 'German_Shepherd', 'Golden_Retriever', 'Labrador_Retriever', 'Poodle', 'Rottweiler', 'Yorkshire_Terrier']
img_size = 32

In [17]:
dog_breed_train_data, dog_breed_train_label = loading_training_data('./DogBreedImageDataset/dataset')

In [37]:
test_data = dog_breed_train_data[0]

In [38]:
test_data.shape

(32, 32, 3)

In [39]:
type(test_data)

numpy.ndarray

In [40]:
# image_np = image_np.astype(np.uint8)  # Ensure it's an unsigned byte array (0-255)

# Convert the NumPy array to a PIL Image
image = Image.fromarray(test_data)

# Define the required transformations for ResNet-50
transform = transforms.Compose([
    transforms.Resize(256),           # Resize to 256x256 (larger than 224x224 to allow for cropping)
    transforms.CenterCrop(224),       # Crop to 224x224
    transforms.ToTensor(),            # Convert to tensor (will scale pixel values to [0, 1])
    transforms.Normalize(             # Normalize the image with ResNet-50 mean and std
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
])

# Apply the transformations
input_tensor = transform(image)

# Add batch dimension (ResNet-50 expects input of shape [batch_size, channels, height, width])
input_tensor = input_tensor.unsqueeze(0)

# Print the shape of the tensor (should be [1, 3, 224, 224])
print(input_tensor.shape)

# Now, you can pass this input_tensor to a model for prediction

torch.Size([1, 3, 224, 224])


In [41]:
model(input_tensor)

tensor([[-9.0025e-01, -8.2897e-01, -2.7587e-01,  9.0487e-02,  1.0005e+00,
          2.3377e-02,  5.0992e-02,  3.5889e-01, -8.0116e-01,  5.9734e-01,
          6.9120e-01, -7.8117e-01, -2.6625e-01,  6.7690e-01,  5.4179e-01,
         -1.2378e+00,  1.3005e+00,  6.6130e-02,  1.0128e-01, -8.4853e-02,
         -1.0930e+00,  2.9924e-01, -3.9452e-01, -2.3037e-01, -5.6824e-01,
          4.5046e-01,  4.2336e-01,  3.3456e-01, -6.8376e-01, -9.7505e-01,
          3.8500e-01,  1.3769e+00,  1.2480e+00, -3.2572e-02, -9.6674e-01,
         -5.3555e-01,  9.5935e-01, -1.5254e-01,  6.5324e-01,  4.5625e-02,
         -1.6542e-02, -4.0806e-01, -4.1751e-01, -1.5022e-01, -1.1263e-01,
          2.0497e-01, -1.4066e-01,  4.6307e-01,  2.7611e-02, -8.7182e-01,
         -4.4118e-01,  2.5375e-03,  2.5510e-02,  5.4150e-01, -9.6354e-02,
          2.4022e-01,  6.9569e-01,  4.2595e-01,  9.1507e-02,  4.2108e-02,
         -3.9851e-01, -2.5334e-01,  6.0019e-02, -7.6522e-01, -9.2369e-01,
         -4.0556e-01,  5.2200e-01, -2.

In [43]:
test_datas = dog_breed_train_data[0:5]

In [49]:
test_datas.shape

(5, 32, 32, 3)

In [46]:
# Example: Load multiple images (let's say 3 images)
# image_paths = ["image1.jpg", "image2.jpg", "image3.jpg"]  # List of image file paths

# Define the transformations
transform = transforms.Compose([
    transforms.Resize(256),           # Resize to 256x256 (larger than 224x224 to allow for cropping)
    transforms.CenterCrop(224),       # Crop to 224x224
    transforms.ToTensor(),            # Convert to tensor (will scale pixel values to [0, 1])
    transforms.Normalize(             # Normalize with ImageNet mean and std
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
])

# Prepare the list of images
input_tensors = []
for image in test_datas:
    # image = Image.open(image_path)  # Open the image
    image = Image.fromarray(image)
    image_tensor = transform(image)  # Apply transformations
    input_tensors.append(image_tensor)

# Stack the images into a single tensor (batch of images)
input_batch = torch.stack(input_tensors)  # Shape will be [batch_size, 3, 224, 224]


In [59]:
input_batch.shape

torch.Size([5, 3, 224, 224])

In [61]:
model(input_batch)

tensor([[-0.6913, -0.2163,  0.6575, -0.2908, -1.0244, -0.2395, -0.0191,  1.4609,
          1.1078,  1.4837],
        [-0.6126, -0.2382,  0.5021, -0.4188, -1.3217, -0.1784, -0.1413,  1.8684,
          1.0650,  1.6394],
        [-0.2959,  0.0656,  0.5339, -0.1434, -0.5605, -0.2595,  0.0629,  1.1306,
          0.6717,  1.1948],
        [-0.5466,  0.1201,  0.6848, -0.2493, -0.6841, -0.1796,  0.0680,  1.2954,
          0.5856,  1.1101],
        [-0.3185,  0.0995,  0.6308, -0.2875, -0.6562, -0.2852, -0.0932,  1.3617,
          0.7612,  1.1964]], grad_fn=<AddmmBackward0>)

In [67]:
import torch
import torch.nn as nn
from torchvision import models

class MultiTaskResNet50(nn.Module):
    def __init__(self, num_classes_task1, num_classes_task2):
        super(MultiTaskResNet50, self).__init__()
        
        # Load the pre-trained ResNet-50 model
        self.resnet = models.resnet50(pretrained=True)
        
        # Remove the final fully connected layer (classifier) for both tasks
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])  # Exclude final FC layer
        
        # Task 1: A new fully connected layer for the first task
        self.fc_task1 = nn.Linear(2048, num_classes_task1)
        
        # Task 2: A new fully connected layer for the second task
        self.fc_task2 = nn.Linear(2048, num_classes_task2)
        
    def forward(self, x):
        # Pass through the ResNet backbone (everything before the final fully connected layer)
        features = self.resnet(x)  # Output size: [batch_size, 2048, 1, 1]
        
        # Flatten the output to [batch_size, 2048]
        features = features.view(features.size(0), -1)
        
        # Task 1: Pass through the task-specific classifier
        task1_output = self.fc_task1(features)
        
        # Task 2: Pass through the task-specific classifier
        task2_output = self.fc_task2(features)
        
        return task1_output, task2_output

# Example usage:
num_classes_task1 = 10  # Number of classes for task 1
num_classes_task2 = 5   # Number of classes for task 2

# Instantiate the model
model = MultiTaskResNet50(num_classes_task1, num_classes_task2)

# Example input tensor with shape [batch_size, 3, 224, 224]
input_tensor = torch.randn(2, 3, 224, 224)  # Batch size of 2

# Forward pass
task1_output, task2_output = model(input_tensor)

print("Task 1 Output:", task1_output.shape)  # Should be [batch_size, num_classes_task1]
print("Task 2 Output:", task2_output.shape)  # Should be [batch_size, num_classes_task2]


Task 1 Output: torch.Size([2, 10])
Task 2 Output: torch.Size([2, 5])
